#Python Advanced Topics

Author: Joe DeRose

Instead of diving deeply into any one particular package, the goal of this session will be to guide you through a few exercises that use packages that you may find useful in your research. 

To start make sure you have the following packages installed:
- scikit-learn
- astropy
- healpy

In [ ]:
%pylab inline
from astropy.table import Table
from astropy.io import fits
from sklearn import datasets, cross_validation, preprocessing, neighbors, metrics, grid_search
from matplotlib.colors import ListedColormap
import healpy as hp

## Reading, writing and manipulating data using astropy
Astropy is a [well documented](http://astropy.readthedocs.org/en/latest/index.html) library containing functionality useful for many every day astrophysical applications. 

First, we'll go over how to load and manipulate FITS files, a common binary data format for astrophysical data.
There are a number of python packages with FITS reading functionality, but today we will focus on astropy.

In [ ]:
hdulist = fits.open('COM_CompMap_CMB-commrul_0256_R1.00.fits')
hdulist.info()

Opening a fits file gives us a list of the 'Header Data Units' or HDUs which contain the data that is stored in the file. The info method lists the properties of each of the HDUs

In [ ]:
hdulist[1].header #We can take a look at the header of each of the HDUs to see what data it contains

We can also write data in FITS format using astropy

In [ ]:
#Let's select the data we care about to write to disk
whdu = hdulist[1]
col1 = whdu.columns['I']
col2 = whdu.columns['VALMASK']

In [ ]:
cols = fits.ColDefs([col1, col2]) #Combine the columns together
tbhdu = fits.BinTableHDU.from_columns(cols) #and turn them into a table
tbhdu.writeto('intensity.fits') #write the table to disk

### Task 1
Read in the file that we just wrote and select the intensity map column.

##Healpy
Now that we've learned how to read and write FITS data using astropy, let's play around with the data we loaded.

[Healpy](https://healpy.readthedocs.org/en/latest/) is a very useful [HEALPix](http://healpix.jpl.nasa.gov/) manipulation package, among other things.  

Briefly, HEALPix is a commonly used decomposition of the sphere. It has useful properties such as:
- All pixels are equal in area
- Pixel centers lie on lines of constant latitude
- [many more](http://healpix.jpl.nasa.gov/pdf/intro.pdf)

A HEALPix map is characterized by its N<sub>side</sub> parameter, describing the number of divisions along the side of a base resolution pixel needed to obtain the desired decomposition. The number of pixels is then N<sub>pix</sub>=4N<sub>side</sub><sup>2</sup>.

In [ ]:
nside = hdulist[1].header['NSIDE']
print(hp.nside2npix(nside))
print(len(hdulist[1].data)) #healpix map is just array w/ N_pix elements (N_side = 256 here)
print(hp.nside2pixarea(nside, degrees=True)) #area of pixels

In [ ]:
#lots of nice coordinate conversion functionality
theta, phi = hp.pix2ang(nside, np.arange(nside)) #get angular locations of pixel centers
vec = hp.ang2vec(theta, phi) #get unit vectors corresponding to those coordinates

In [ ]:
colormap = ListedColormap(np.loadtxt("Planck_Parchment_RGB.txt")/255.) #to make it pretty
hp.mollview(hdulist[1].data['I'], nest=True, cmap=colormap) #nested numbering scheme 

Healpix has some convinient coordinate transformation functions.

In [ ]:
euler_angs = [45, 45, 0] #we can define an arbitraty rotation using Euler angles
rot = hp.Rotator(euler_angs, eulertype='XYZ')
print(rot.mat) #the rotation matrix in cartesian basis defined by euler_angs

In [ ]:
hp.mollview(hdulist[1].data['I'], nest=True, cmap=colormap, rot=euler_angs)
hp.graticule() #plot the graticule

In [ ]:
hp.mollview(hdulist[1].data['I'], nest=True, cmap=colormap, coord=['G', 'E'])
hp.graticule()

In [ ]:
mmap = hp.ma(hdulist[1].data['I']) 
mmap.mask = np.logical_not(hdulist[1].data['VALMASK']) #we can mask out the galactic plane
hp.mollview(mmap, nest=True, cmap=colormap, coord=['G', 'E'])

Astropy has some very convenient coordinate transformation functionality as well.

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

ra = np.arange(10)
dec = np.arange(10)
c = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
c = SkyCoord(ra, dec, frame='icrs', unit='deg') #alternatively
c

In [ ]:
c.galactic #to convert between different coordinate systems, just get the relevant attributes

### Task 2
Mask out all pixels with RA>90 in the CMB map and plot them using the Mollweide projection.

##Scikit-learn
There is [a lot](http://scikit-learn.org/stable/) here, so we'll only just scratch the surface. There are a very large number of machine learning algorithms implemented in this package. I'll use just one for the purposes of this tutorial, focusing on a typical model building workflow.

We'll cover:
- Data pre-processing and feature selection
- Model selection/validation
- Model persistence

In [ ]:
boston = datasets.load_boston() #load in a canned data set
X = boston.data
Y = boston.target
fields = boston.feature_names
print(boston.DESCR)

Let's take a look!

In [ ]:
subplot(131)
scatter(X[:,11],Y)
subplot(132)
scatter(X[:,10],Y)
subplot(133)
scatter(X[:,9],Y)
tight_layout()

Note that the dynamical range of some of the features is much larger than that of others. Depending on the learning algorithm, this can lead to non-optimal performance, so it is always good practice to rescale your features.

In [ ]:
X_scaled = preprocessing.scale(X)
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X_scaled, Y, test_size=0.5)

We'll try to predict the price of the homes based on the other features based on a k-nearest neighbors algorithm. The most basic version of this algorithm predicts the label for the query point as the mean of the labels of the k nearest neighbors to the query point in feature space.

In [ ]:
reg = neighbors.KNeighborsRegressor(5)
reg.fit(X_train, Y_train)

In [ ]:
Y_pred = reg.predict(X_test)

# how well did we do?
mse = metrics.mean_squared_error(Y_test,Y_pred)
print(mse)
plot(Y_test,Y_pred - Y_test,'o')
xlabel("True Median House Price ($1,000)")
ylabel("Residual")
hlines(0,min(Y_test),max(Y_test),color="red")

### Task 3

Explore the KNN model parameters. See if you can find a combination that minimzes the MSE.

We can do this more systematically using the GridSearchCV sklearn function!

In [ ]:
params = {'n_neighbors':range(1,5), 'weights':['uniform', 'distance']}
cvreg = grid_search.GridSearchCV(neighbors.KNeighborsRegressor(), params, n_jobs=-1)
cvreg.fit(X_train, Y_train)

In [ ]:
cvreg.grid_scores_

In [ ]:
Y_pred = cvreg.predict(X_test)

# how well did we do?
mse = metrics.mean_squared_error(Y_test,Y_pred)
print(mse)
plot(Y_test,Y_pred - Y_test,'o')
xlabel("True Median House Price ($1,000)")
ylabel("Residual")
hlines(0,min(Y_test),max(Y_test),color="red")